# **（1）几何练习**


In [1]:
# 安装
%pip install duckdb leafmap

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 515.2/515.2 kB 7.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 213.7/213.7 kB 15.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.1/6.1 MB 58.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.9/41.9 kB 2.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 108.6/108.6 kB 7.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.7/2.7 MB 30.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 717.6/717.6 kB 26.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.2/194.2 kB 13.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.4/72.4 kB 4.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 51.9 MB/s eta 0:00:00


In [2]:
# 导入库
import duckdb
import leafmap

In [3]:
# 数据
url = "https://open.gishub.org/data/duckdb/nyc_data.db.zip"
leafmap.download_file(url, unzip=True)

Downloading...
From: https://open.gishub.org/data/duckdb/nyc_data.db.zip
To: /content/nyc_data.db.zip
100%|██████████| 8.60M/8.60M [00:00<00:00, 25.1MB/s]


Extracting files...


'/content/nyc_data.db.zip'

In [4]:
# 连接到duckDBdb
con = duckdb.connect("nyc_data.db")

In [5]:
con.install_extension("spatial")
con.load_extension("spatial")

In [6]:
con.sql("SHOW TABLES;")

┌─────────────────────┐
│        name         │
│       varchar       │
├─────────────────────┤
│ nyc_census_blocks   │
│ nyc_homicides       │
│ nyc_neighborhoods   │
│ nyc_streets         │
│ nyc_subway_stations │
└─────────────────────┘

1.What is the area of the 'West Village' neighborhood?
  “西村”街区的面积是多少？（提示：面积以平方米为单位。要得到以公顷为单位的面积，请除以 10000。要得到以英亩为单位的面积，请除以 4047。）

In [7]:
con.sql("SELECT * from nyc_neighborhoods LIMIT 5")

┌───────────┬──────────────────────────┬────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────

In [10]:
con.sql("""
SELECT
    ST_Area(geom) AS area_in_square_meters,
    ST_Area(geom) / 10000 AS area_in_hectares,
    ST_Area(geom) / 4047 AS area_in_acres
FROM
    nyc_neighborhoods
WHERE
    NAME = 'West Village'
""")

┌───────────────────────┬────────────────────┬────────────────────┐
│ area_in_square_meters │  area_in_hectares  │   area_in_acres    │
│        double         │       double       │       double       │
├───────────────────────┼────────────────────┼────────────────────┤
│    1044614.5296485956 │ 104.46145296485956 │ 258.12071402238587 │
└───────────────────────┴────────────────────┴────────────────────┘

2. What is the geometry type of ‘Pelham St’? The length?

In [17]:
con.sql("SELECT * from nyc_streets LIMIT 2")

┌───────┬─────────────┬─────────┬─────────────┬────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────┐
│  ID   │    NAME     │ ONEWAY  │    TYPE     │                                                                                                                                                                                                                                                                          geom                                                                                                  

In [18]:
con.sql("""
SELECT
    ST_GeometryType(geom) AS geometry_type,
    ST_Length(geom) AS length
FROM
    nyc_streets
WHERE
    NAME = 'Pelham St';
""")

┌─────────────────┬───────────────────┐
│  geometry_type  │      length       │
│  geometry_type  │      double       │
├─────────────────┼───────────────────┤
│ MULTILINESTRING │ 50.32314951660229 │
└─────────────────┴───────────────────┘

3. What is the GeoJSON representation of the 'Broad St' subway station?

In [19]:
con.sql("""
SELECT
    NAME,
    ST_AsGeoJSON(geom) AS geojson
FROM
    nyc_subway_stations
WHERE
    NAME = 'Broad St';
    """)

┌──────────┬──────────────────────────────────────────────────────────────────────┐
│   NAME   │                               geojson                                │
│ varchar  │                                 json                                 │
├──────────┼──────────────────────────────────────────────────────────────────────┤
│ Broad St │ {"type":"Point","coordinates":[583571.9059213118,4506714.341192182]} │
└──────────┴──────────────────────────────────────────────────────────────────────┘

4. What is the total length of streets (in kilometers) in New York City? (Hint: The units of measurement of the spatial data are meters, there are 1000 meters in a kilometer.)

In [20]:
con.sql("""
SELECT
    SUM(ST_Length(geom)) / 1000 AS total_length_km
FROM
    nyc_streets;
    """)

┌────────────────────┐
│  total_length_km   │
│       double       │
├────────────────────┤
│ 10418.904717199997 │
└────────────────────┘

5. What is the area of Manhattan in acres英亩? (Hint: both nyc_census_blocks and nyc_neighborhoods have a boroname in them.)

In [21]:
con.sql("SELECT * from nyc_census_blocks LIMIT 5")

┌─────────────────┬────────────┬────────────┬────────────┬────────────┬────────────┬────────────┬───────────────┬─────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────┐
│      BLKID      │ POPN_TOTAL │ POPN_WHITE │ POPN_BLACK │ POPN_NATIV │ POPN_ASIAN │ POPN_OTHER │   BORONAME    │               

In [22]:
con.sql("""
SELECT
    SUM(ST_Area(geom)) * 0.000247105 AS area_acres
FROM
    nyc_census_blocks
WHERE
    BORONAME = 'Manhattan';
    """)

┌────────────────────┐
│     area_acres     │
│       double       │
├────────────────────┤
│ 14601.894220020411 │
└────────────────────┘

6. What is the most westerly subway station?

In [23]:
con.sql("""
SELECT
    NAME,
    ST_AsText(geom) AS geometry
FROM
    nyc_subway_stations
ORDER BY
    ST_X(geom) ASC
LIMIT 1;
    """)

┌─────────────┬─────────────────────────────────────────────┐
│    NAME     │                  geometry                   │
│   varchar   │                   varchar                   │
├─────────────┼─────────────────────────────────────────────┤
│ Tottenville │ POINT (563292.1172580556 4484900.921251608) │
└─────────────┴─────────────────────────────────────────────┘

7. How long is 'Columbus Cir' (aka Columbus Circle)?

In [26]:
con.sql("""
SELECT *
FROM nyc_streets
WHERE NAME ILIKE '%Columbus Cir%';
""")

┌───────┬──────────────┬─────────┬─────────┬────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────

In [27]:
con.sql("""
SELECT
    NAME,
    ST_Length(geom) AS length
FROM
    nyc_streets
WHERE
    NAME ILIKE '%Columbus Cir%';
""")

┌──────────────┬───────────────────┐
│     NAME     │      length       │
│   varchar    │      double       │
├──────────────┼───────────────────┤
│ Columbus Cir │ 308.3419936909855 │
└──────────────┴───────────────────┘

7. What is the length of streets in New York City, summarized by type?

In [30]:
con.sql("""
SELECT
    TYPE,
    SUM(ST_Length(geom)) AS total_length
FROM
    nyc_streets
GROUP BY
    TYPE
ORDER BY
    total_length DESC;
""")

┌──────────────────────────────────────────────────┬────────────────────┐
│                       TYPE                       │    total_length    │
│                     varchar                      │       double       │
├──────────────────────────────────────────────────┼────────────────────┤
│ residential                                      │   8629870.33786606 │
│ motorway                                         │  403622.4781263628 │
│ tertiary                                         │  360394.8790513027 │
│ motorway_link                                    │   294261.419479668 │
│ secondary                                        │  276264.3038979258 │
│ unclassified                                     │  166936.3716044583 │
│ primary                                          │  135034.2330179469 │
│ footway                                          │  71798.48783780965 │
│ service                                          │ 28337.635038596007 │
│ trunk                               

# （2）Spatial Relationships Exercises

1. What is the geometry value for the street named 'Atlantic Commons'?

In [31]:
con.sql("""
SELECT
    NAME,
    geom
FROM
    nyc_streets
WHERE
    NAME ILIKE 'Atlantic Commons';
""")

┌──────────────────┬──────────────────────────────────────────────────────────────────────────────────────────────┐
│       NAME       │                                             geom                                             │
│     varchar      │                                           geometry                                           │
├──────────────────┼──────────────────────────────────────────────────────────────────────────────────────────────┤
│ Atlantic Commons │ MULTILINESTRING ((586781.7015777241 4504202.153143394, 586863.5196448397 4504215.988170098)) │
└──────────────────┴──────────────────────────────────────────────────────────────────────────────────────────────┘

2. What neighborhood and borough is Atlantic Commons in?

In [33]:
con.sql("SELECT * from nyc_neighborhoods LIMIT 5")

┌───────────┬──────────────────────────┬────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────

In [34]:
con.sql("""
SELECT
    n.NAME,
    n.BORONAME
FROM
    nyc_streets s
JOIN
    nyc_neighborhoods n
ON
    ST_Intersects(s.geom, n.geom)  -- 假设使用几何相交来确定位置
WHERE
    s.NAME ILIKE 'Atlantic Commons';
""")

┌────────────┬──────────┐
│    NAME    │ BORONAME │
│  varchar   │ varchar  │
├────────────┼──────────┤
│ Fort Green │ Brooklyn │
└────────────┴──────────┘

3. What streets does Atlantic Commons join with?

In [35]:
con.sql("""
SELECT
    s2.NAME AS intersecting_street
FROM
    nyc_streets s1
JOIN
    nyc_streets s2
ON
    ST_Intersects(s1.geom, s2.geom)
WHERE
    s1.NAME ILIKE 'Atlantic Commons';
""")

┌─────────────────────┐
│ intersecting_street │
│       varchar       │
├─────────────────────┤
│ S Oxford St         │
│ Atlantic Commons    │
│ Cumberland St       │
└─────────────────────┘

4. Approximately how many people live on (within 50 meters of) Atlantic Commons?

In [36]:
con.sql("""
SELECT
    SUM(c.POPN_TOTAL) AS total_population
FROM
    nyc_census_blocks c
JOIN
    nyc_streets s
ON
    ST_DWithin(c.geom, s.geom, 50)  -- 检查人口几何是否在街道几何50米范围内
WHERE
    s.NAME ILIKE 'Atlantic Commons';
""")

┌──────────────────┐
│ total_population │
│      int128      │
├──────────────────┤
│             1438 │
└──────────────────┘